In [17]:
import pandas as pd

def build_dir_list(dir_path):
    # 文件地址列表
    list_file_path = []
    # 文件名及后缀名的分割列表
    list_spli_file = []
    for file in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file)
        if os.path.isfile(file_path):
            list_file_path.append(file_path)
            spli_file = os.path.splitext(file)
            list_spli_file.append(spli_file)

    df = pd.DataFrame({"file_path":list_file_path, "spli_file":list_spli_file})

    df['name'] = df['spli_file'].apply(lambda x: x[0])
    df['type'] = df['spli_file'].apply(lambda x: x[1])

    sup_df = df.duplicated(subset='name', keep=False)
    sup_df = pd.DataFrame({"flag":sup_df})

    # join方式合并
    outer_df_join = df.join(sup_df, how='outer')
    # merge方式合并
    # outer_df_merge = pd.merge(sup_df, df, left_index=True, right_index=True, how='outer')

    return outer_df_join

# outer_df_join = build_dir_list("F:\HEIC\新建文件夹")

In [18]:
import time, os.path

def get_file_updatetime(file_path):
    update_mtime = os.path.getmtime(file_path)
    x = time.localtime(update_mtime)
    update_time_format = time.strftime("%Y%m%d", x)
    
    return update_time_format

def rename_jpg_name(file_path_jpg, file_path_heic, spli_file_jpg, target_dir_path):
    # 获取heic文件的修改时间（%Y%m%d）
    heic_updatetime = get_file_updatetime(file_path_heic)
    new_file_name = os.path.join(spli_file_jpg[0] + '_' + heic_updatetime + spli_file_jpg[1])
    # 拼接新的jpg文件名为：target_dir_path + oldname + _ + %Y%m%d + .jpg
    new_file_path = os.path.join(target_dir_path, new_file_name)
    # 修改jpg文件名为新文件名
    os.rename(file_path_jpg, new_file_path)
    # 删除heic文件
    os.remove(file_path_heic)
    print("删除%s,修改%s成功!" % (file_path_heic,new_file_path))

# print(get_file_updatetime(r"F:\HEIC\main\111.txt"))

# rename_jpg_name(r"F:\HEIC\demo\IMG_5310.jpg", r"F:\HEIC\demo\IMG_5310.HEIC", ['IMG_5310','.jpg'],  r"F:\HEIC\demo")

第一版本：
1、利用第三方工具将heic文件格式转换为同名的jpg文件
2、人工筛选jpg文件，删除无用文件，剩余需要保留文件
3、将全量heic文件及需要保留的jpg文件拷贝至同一目录下
4、通过pandas筛选出jpg与heic名称一致的文件
5、不重复的表示已手动删除jpg的无用文件，执行删除
6、获取剩余的heic文件的修改时间并格式化%Y%m%d，拼接到与heic名称一致的jpg文件之后 name + %Y%m%d + .jpg
7、删除全部heic文件


In [19]:
"""
1、全量heci文件
2、需要保留的jpg文件
3、拷贝至同一目录PATH下
"""

PATH = "F:\HEIC\新建文件夹"
DELETE_TYPE = ".HEIC"
MAIN_TYPE = '.jpg'

def transform_heic01():
    # 获取指定目录下的文件列表
    outer_df_join = build_dir_list(PATH)

    # 删除不重复的无用的.HEIC文件(保留jpg与heic均存在的文件)
    data_useless = outer_df_join[(outer_df_join['flag'] == False) & (outer_df_join['type'] == DELETE_TYPE)]
    data_useless['file_path'].apply(lambda x: os.remove(x))

    # 将heic与jpg文件信息通过name拼接，方便处理
    data_same = outer_df_join[(outer_df_join['flag'] == True ) & ((outer_df_join['type'] == DELETE_TYPE) | (outer_df_join['type'] == MAIN_TYPE))] 
    data_same_heic = data_same[data_same['type'] == DELETE_TYPE]
    data_same_jpg = data_same[data_same['type'] == MAIN_TYPE]
    data_merger = pd.merge(data_same_heic, data_same_jpg, on='name', suffixes=('_heic','_jpg'), how='outer')
    # 根据.HEIC文件修改时间，修改jpg文件的名字
    for index, row in data_merger.iterrows():
        rename_jpg_name(row['file_path_jpg'], row['file_path_heic'], row['spli_file_jpg'], PATH)
    print("=================== success =========================")


第二版
1、利用第三方工具将heic文件格式转换为同名的jpg文件
2、将全量的heic与jpg文件拷贝至同一目录下
3、将jpg文件名重命名为，拼接上对应name的heic文件的修改时间，name + %Y%m%d + .jpg
4、删除全部heic文件，人工筛选出需要保留的jpg文件

In [38]:

import math

DELETE_TYPE = ".HEIC"
MAIN_TYPE = '.jpg'

def rename_and_remove(dir_path):
    print("开始执行修改文件名操作")
    # 获取指定目录下的文件列表
    outer_df_join = build_dir_list(dir_path)
    data_same_heic = outer_df_join[outer_df_join['type'] == DELETE_TYPE]
    data_same_jpg = outer_df_join[outer_df_join['type'] == MAIN_TYPE]
    data_merger = pd.merge(data_same_heic, data_same_jpg, on='name', suffixes=('_heic','_jpg'), how='outer')
    # 删除NaN行(所有字段均不能为NaN)
    data_result = data_merger.dropna(inplace=False)  
    # 根据.HEIC文件修改时间，修改jpg文件的名字
    for index, row in data_result.iterrows():
        rename_jpg_name(row['file_path_jpg'], row['file_path_heic'], row['spli_file_jpg'], dir_path)
    print("=================== success =========================")

# 将指定文件夹下的jpg文件根据heic的修改时间命名
# rename_and_remove(r"F:\HEIC\新建文件夹8\新建文件夹")

开始执行修改文件名操作
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8297.HEIC,修改F:\HEIC\新建文件夹8\新建文件夹\IMG_8297_20220329.jpg成功!
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8298.HEIC,修改F:\HEIC\新建文件夹8\新建文件夹\IMG_8298_20220329.jpg成功!
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8299.HEIC,修改F:\HEIC\新建文件夹8\新建文件夹\IMG_8299_20220329.jpg成功!
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8300.HEIC,修改F:\HEIC\新建文件夹8\新建文件夹\IMG_8300_20220329.jpg成功!
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8301.HEIC,修改F:\HEIC\新建文件夹8\新建文件夹\IMG_8301_20220329.jpg成功!
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8302.HEIC,修改F:\HEIC\新建文件夹8\新建文件夹\IMG_8302_20220329.jpg成功!
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8303.HEIC,修改F:\HEIC\新建文件夹8\新建文件夹\IMG_8303_20220329.jpg成功!
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8304.HEIC,修改F:\HEIC\新建文件夹8\新建文件夹\IMG_8304_20220329.jpg成功!
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8305.HEIC,修改F:\HEIC\新建文件夹8\新建文件夹\IMG_8305_20220329.jpg成功!
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8306.HEIC,修改F:\HEIC\新建文件夹8\新建文件夹\IMG_8306_20220329.jpg成功!
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8307.HEIC,修改F:\HEIC\新建文件夹8\新建文件夹\IMG_8307_20220329.jpg成功!
删除F:\HEIC\新建文件夹8\新建文件夹\IMG_8308.HEIC,修改F:\HEIC\新建文件夹8

In [31]:
import os

SHARE_USER = 'commiao'
SHARE_PASSWORD = 'han880924'
SHARE_FILE_HOST = r'\\192.168.31.99'

def copy_share_file_commiao(share_file_path, target_file_path):
    print("开始操作NAS-commiao")
    permission_cmd = f'net use {SHARE_FILE_HOST} {SHARE_USER} /user:{SHARE_PASSWORD}'
    print("初始化共享文件夹permission_cmd:%s" % permission_cmd)
    print("开始copy共享文件%s, 目标文件%s" % (share_file_path, target_file_path))
    permission_result = os.popen(permission_cmd)
    print(permission_result.read())
    copy_cmd = f'xcopy /y {share_file_path} {target_file_path}'
    print("copy共享文件操作结束:%s" % copy_cmd)
    copy_result = os.popen(copy_cmd)
    print(copy_result.read())

In [33]:
PATH = r'F:\HEIC'
file_name = '新建文件夹9'
target_dir_path = os.path.join(PATH, file_name)
share_path = r'\\192.168.31.99\photo\hanminmin'

share_path_todo = os.path.join(share_path, 'todo', file_name)
copy_share_file_commiao(share_path_todo, target_dir_path)

share_path_heic = os.path.join(share_path, 'heic', file_name)
copy_share_file_commiao(share_path_heic, target_dir_path)

rename_and_remove(target_dir_path)


开始操作NAS-commiao
初始化共享文件夹permission_cmd:net use \\192.168.31.99 commiao /user:han880924
开始copy共享文件\\192.168.31.99\photo\hanminmin\todo\新建文件夹9, 目标文件F:\HEIC\新建文件夹9

copy共享文件操作结束:xcopy /y \\192.168.31.99\photo\hanminmin\todo\新建文件夹9 F:\HEIC\新建文件夹9
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9077.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9268.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9509_1.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9487_2.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9123_1.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9538.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9517_1.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9708_1.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9698.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9547.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9538_2.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9410_1.jpg
\\192.168.31.99\photo\hanminmin\todo\新建文件夹9\IMG_9386_1.jpg


In [36]:
file_name = '新建文件夹9'
share_path = r'\\192.168.31.99\photo\hanminmin'

share_path_todo = os.path.join(share_path, 'todo', file_name)
print(share_path_todo)
share_path_heic = os.path.join(share_path, 'heic', file_name)
print(share_path_heic)

\\192.168.31.99\photo\hanminmin\todo\新建文件夹9
\\192.168.31.99\photo\hanminmin\heic\新建文件夹9
